In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp

def optimizar_lunes_seguro():
    print("--- ESTRATEGIA LUNES: MAX SHARPE (CON LÍMITE 35%) ---")
    
    # 1. Cargar datos
    try:
        df = pd.read_csv('prod_long_sharpe_u50_20260116_v5_train_dataset.csv')
    except:
        print("Error: No se encuentra el csv.")
        return

    returns = df.values
    assets = df.columns
    mu = np.mean(returns, axis=0)          
    Sigma = np.cov(returns, rowvar=False)  
    rf = 0.0
    n_total = len(assets)

    # ----------------------------------------------------
    # PASO 1: Identificar los Top 5 Activos (Pre-selección)
    # ----------------------------------------------------
    # Hacemos una pasada rápida de Max Sharpe sin límites
    y_pre = cp.Variable(n_total)
    risk_pre = cp.quad_form(y_pre, Sigma)
    prob_pre = cp.Problem(cp.Minimize(risk_pre), 
                          [(mu - rf) @ y_pre == 1, y_pre >= 0])
    prob_pre.solve(solver=cp.SCS)
    
    # Seleccionamos los índices de los 5 mayores pesos
    w_pre = y_pre.value
    top_5_indices = np.argsort(w_pre)[-5:] # Índices de los 5 mejores
    
    print("Activos seleccionados para la cartera:", assets[top_5_indices].values)

    # ----------------------------------------------------
    # PASO 2: Optimizar con Límite del 35%
    # ----------------------------------------------------
    # Ahora solo trabajamos con esos 5 activos
    mu_5 = mu[top_5_indices]
    Sigma_5 = Sigma[np.ix_(top_5_indices, top_5_indices)]
    
    # Nueva variable solo para los 5 elegidos
    y_5 = cp.Variable(5)
    
    # Función objetivo igual (Minimizar Riesgo sujeto a Retorno 1)
    risk_5 = cp.quad_form(y_5, Sigma_5)
    
    constraints_5 = [
        (mu_5 - rf) @ y_5 == 1,         # Normalización de retorno
        y_5 >= 0,                       # Long only
        # RESTRICCIÓN CLAVE: Peso individual <= 35% del total
        # En Charnes-Cooper esto se escribe como: y_i <= 0.35 * sum(y)
        y_5 <= 0.35 * cp.sum(y_5)
    ]

    prob_5 = cp.Problem(cp.Minimize(risk_5), constraints_5)
    try:
        prob_5.solve()
    except:
        prob_5.solve(solver=cp.SCS)

    # ----------------------------------------------------
    # PASO 3: Generar Cadena Final
    # ----------------------------------------------------
    w_final_5 = y_5.value / np.sum(y_5.value)
    
    # Construir el vector de 50 activos lleno de ceros
    w_total = np.zeros(n_total)
    # Rellenar solo las posiciones de los Top 5
    w_total[top_5_indices] = w_final_5
    
    # Formatear cadena
    lista_str = []
    for w in w_total:
        val = round(w, 3)
        # Asegurar formato "0.0" o "0.35"
        if val == 0:
            lista_str.append("0.0")
        else:
            lista_str.append(str(val))
            
    print("\n>>> COPIA ESTA CADENA: <<<")
    print("-" * 50)
    print(" ".join(lista_str))
    print("-" * 50)
    
    # Mostrar distribución para tu tranquilidad
    print("\nDistribución de pesos (debe ser máx 0.35):")
    for i, idx in enumerate(top_5_indices):
        print(f"{assets[idx]}: {w_final_5[i]:.4f}")

if __name__ == "__main__":
    optimizar_lunes_seguro()

--- ESTRATEGIA LUNES: MAX SHARPE (CON LÍMITE 35%) ---
Activos seleccionados para la cartera: ['asset2' 'asset8' 'asset23' 'asset36' 'asset6']

>>> COPIA ESTA CADENA: <<<
--------------------------------------------------
0.0 0.0 0.0 0.0 0.0 0.35 0.0 0.086 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.218 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.346 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
--------------------------------------------------

Distribución de pesos (debe ser máx 0.35):
asset2: 0.0000
asset8: 0.0865
asset23: 0.2178
asset36: 0.3457
asset6: 0.3500
